In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai chromadb gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 35.0 MB/s eta 0:00:

In [2]:
sql_path = '/content/drive/MyDrive/chatbot_sql/products.db'

In [3]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"sqlite:////{sql_path}")
print(db.dialect)
print(db.get_usable_table_names())
db.run("select * from products limit 10")


sqlite
['categories', 'products']


"[('lazadacps_476_4925258281_lazada.vn', 'Máy nấu sáp wax nóng pro 100 tẩy lông chuyên dụng nồi nấu sáp pro 100 tặng kèm sáp và que gỗ', 'Máy nấu sáp wax nóng pro 100 tẩy lông chuyên dụng nồi nấu sáp pro 100 tặng kèm 1 gói sáp 100 gram và que gỗ♥️ Máy nấu sáp wax nóng pro 100 tẩy lông chuyên dụng nồi nấu sáp pro 100 tặng kèm 1 gói sáp 100 gram mùi ngẫu nhiên và que gỗ. Mua nồi bên mình sẽ tặng kèm 100 gram sáp liddy và 10 que gỗ ....', 17, 130779.39, 74), ('cavailles_shopee_322_5836032590_shopee.vn', 'Lốp (Vỏ) xe đạp Deli Tire S601 27.5x1.75 - 1 Cái', 'Lốp (Vỏ) xe đạp Deli Tire S601 27.5x1.75 - 1 Cái\\n- Nhãn hiệu: Deli Tire\\n- Model: S601\\n- Kích cỡ lốp 27.5x1.75\\n- Chất liệu: cao su\\n- Rim type : SS\\n- Etrto (mm) : 47 - 584\\n- PSI : 35 - 65\\n- Bar : 2.4 - 4.5\\n- Trọng lượng ~ 530g\\n  - Màu sắc: Đen\\nMade in Indonesia\\n-----------------\\n▶ XE ĐẠP PHƯỢNG...', 24, 53226.14, 64), ('tikivn_62_224005697_tiki.vn', 'Bộ 2 Chai Dung Dịch Tẩy Keo, Nhựa Đường 3M 08987 425 g', 'Dung D

In [ ]:
# select * from products where name = 'Máy nấu sáp wax nóng pro 100 tẩy lông chuyên dụng nồi nấu sáp pro 100 tặng kèm sáp và que gỗ'

In [5]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

#Few shot

## Example query

In [6]:
examples = [
    {
        "input": "Danh mục sản phẩm có những gì?",
        "query": """
        select id, name from categories
        """
    }
]

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(model='text-embedding-3-small'),
    Chroma,
    k=1,
    input_keys=['input']
)


In [10]:
example_selector.select_examples({"input" : "Sản phẩm nào dưới 300 nghìn"})

[{'query': '\n        select id, name from categories\n        ',
  'input': 'Danh mục sản phẩm có những gì?'}]

# Corect product name

In [11]:
from langchain.agents.agent_toolkits import create_retriever_tool

In [13]:
import ast


def query_as_list(db, query):
  res = db.run(query)
  res = [el for sub in ast.literal_eval(res) for el in sub if el]
  return list(set(res))

product = query_as_list(db, "select name from products") + query_as_list(db, "select name from categories")
print(product[:-10])


['Đồng Hồ Nữ Halei Dáng Lắc Mặt Tròn Dây Vàng Mặt Trắng', 'Túi trám ô size mini 15cm kèm hộp fullbox', 'Sét Bộ thu đông SuMe Basic Nỉ cotton loại 1 Nam Nữ Ulzzang tay bồng kèm quần Bon Chun ống🍀Đồ mặc nhà ngủ; đi chơi', 'Túi Khoá đồng tròn khuy gấp size 20cm KO hộp', 'Larita Manzo AH0724_24 M Silver', 'Củ Sạc Điện Thoại Ô Tô 40W, 2 Cổng Type-C PD 20W CHOETECH TC0008 V2 - Hàng Chính Hãng', 'Áo thun tay lỡ Unisex, áo phông form rộng', ' Giày nữ thời trang ELLY- EG155 ', 'Bộ chế hòa khí (BXC) HONDA Wave anpha nhỏ đời đầu  QA(16100KRS901)', 'Túi treo khung sườn hộp cứng xe đạp NEWBOLER có ngăn điện thoại', 'Túi Dây xích Ngọc hộp nơ xịn sò xuất xịn size mini 18cm', 'Tẩy tế bào chết body YZKMSKIN', 'Dây ga B HONDA Vision 110 2021+  QA(17920K2CV01)', 'Quần dài màu đỏ cho bé gái', '{Nhùn nhùn}Áo Choàng Dokma nỉ trần bông đa năng có mũ dành cho bé trai bé gái 1-2y và 2-3y', 'Bộ ốp trước dưới trái (Nh187R) HONDA Air Blade 2012-2014  QA(64520K27V00ZE)', 'QUẦN BAGGY JEAN NỮ CẠP CAO ( VIDEO - ẢNH T

In [14]:
vector_db = Chroma.from_texts(product, OpenAIEmbeddings(model='text-embedding-3-small'))
retriever = vector_db.as_retriever(search_kwargs={"k" : 15})
description = "Use to look up values to filter on. Input is an approxiamte spelling of the proper noun, output is valid proper nouns. Use the noun most similar to the search"

retriever_tool = create_retriever_tool(
    retriever,
    description=description,
    name="search_proper_nouns"
)

In [15]:
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate, MessagesPlaceholder, PromptTemplate

In [67]:
def get_answer(user_query):
  top_k = 20
  dialect = db.dialect

  system_prefix = f"""
  You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:
"""
  # print(system_prefix)
  ##fall_back
  fallback_example = [
    {
        "input": "Danh mục sản phẩm có những gì?",
        "query": """
        select id, name from categories
        """
    }
  ]

  try:
    safe_examples = example_selector.select_examples({"input" : user_query})
    example_prompt = PromptTemplate.from_template("User input: {input}\n SQL query: {query}")
    for ex in safe_examples:
      if "input" not in ex or "query" not in ex:
        raise ValueError("Every example must contain an input and query key.")
    few_shot_prompt = FewShotPromptTemplate(
      example_selector=example_selector,
      example_prompt=example_prompt,
      prefix=system_prefix,
      suffix="",
      input_variables=["input"],
    )

    # print(few_shot_prompt)
  except Exception as e:
    # print("Fall_back")
    few_shot_prompt = FewShotPromptTemplate(
      examples=fallback_example,
      example_prompt=example_prompt,
      prefix=system_prefix,
      suffix="",
      input_variables=["input"],
    )

  prompt_val = few_shot_prompt.format(input=user_query)

  # print(prompt_val)

  system_unique_name_prompt = f"""
  If you need to filter on a proper noun, you must Always first look up the filter value using the "search_proper_nouns" tool!
  You have to access to the following tables: {', '.join(db.get_usable_table_names())}

  If the question doess not seem related to the database, just return "I don't know" as the answer.
  """
  final_prompt = prompt_val + "\n" + system_unique_name_prompt

  full_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", final_prompt),
          ("human", "{input}"),
          MessagesPlaceholder("agent_scratchpad"),
      ]
  )

  # print(final_prompt)

  agent = create_sql_agent(
      llm=llm,
      db = db,
      max_iterations=40,
      extra_tools=[retriever_tool],
      prompt=full_prompt,
      agent_type="openai-tools",
      verbose=True
  )

  result = agent.invoke({"input": user_query})
  return result['output']

In [93]:
result = get_answer("Giá của sản phẩm ' Lốp (Vỏ) xe đạp Deli'")



> Entering new SQL Agent Executor chain...

Invoking: `search_proper_nouns` with `{'query': 'Lốp (Vỏ) xe đạp Deli'}`


Lốp (Vỏ) xe đạp Deli Tire S601 27.5x1.75 - 1 Cái

Xe Đạp

Đầu van bơm bình khí CO2 xe đạp ThinkRider

Pô tăng xe đạp TRUVATIV cỡ 25.4x32/60/80/90mm

Lắp ráp xếp hình lego city 5512 : Xe Đua thể thao (Khách hàng chat chọn mẫu)

Chén cổ (bát phốt) thả xe đạp 34mm

Bu lông HONDA 10x155 dùng chung nhiều xe  QA(90101K28910) _( 155)

Chân chống nhôm xe đạp kẹp 2 bách tăng giảm tem đỏ

Khuôn silicon hình xe ô tô to làm thạch rau câu, pudding, mousse khuôn oto khuôn tàu hỏa

Đồ Chơi

Phớt O-RING HONDA 74.5X2.5 Future 125 / Fi  QA(91301PH8005) _( 230)

Trục giữa cốt rỗng xe đạp SHIMANO DEORE SM BB52 (BB VẶN) - Hàng Chính Hãng

Giày Dép Nam

Bộ ốp trước dưới trái (Nh187R) HONDA Air Blade 2012-2014  QA(64520K27V00ZE)

Giày Dép Nữ
Invoking: `sql_db_query_checker` with `{'query': "select price from products where name = 'Lốp (Vỏ) xe đạp Deli Tire S601 27.5x1.75 - 1 Cái'"}`


```s

In [69]:
result

'Dưới đây là 5 sản phẩm có giá cao nhất:\n\n1. **Laptop Lenovo IdeaPad Slim 3 15ABR8 (82XM00EHVN)** - Giá: 1,487,865.85 VNĐ\n2. **Laptop Lenovo IdeaPad Slim 3 15ABR8 (82XM00EHVN)** - Giá: 1,487,865.85 VNĐ\n3. **[HCM] Sữa tắm Dettol Kháng khuẩn và dưỡng thể (Cam) chai 950g** - Giá: 1,485,350.63 VNĐ\n4. **[HCM] Sữa tắm Dettol Kháng khuẩn và dưỡng thể (Cam) chai 950g** - Giá: 1,485,350.63 VNĐ\n5. **Thảo Dược Xông Hơ Vùng Kín Cho Mẹ Sau Sinh** - Giá: 1,478,713.76 VNĐ\n\nLưu ý rằng một số sản phẩm có giá giống nhau.'

#UI

In [90]:
def chat(user_message, history):
  if history is None:
    history = []

  message = history.copy()

  message.append({"role": "user", "content": user_message})

  result = get_answer(str(message))

  bot_reply = result

  history.append({"role": "user", "content": user_message})
  history.append({"role": "assistant", "content": bot_reply})

  return "", history

In [91]:
import gradio as gr

In [92]:
with gr.Blocks() as demo:
  gr.Markdown("Chatbot SQL")
  chatbot = gr.Chatbot(type = "messages")
  with gr.Row():
    txt = gr.Textbox(
        show_label=False,
        placeholder="Nhập tin nhắn...",
    )
  txt.submit(chat, [txt, chatbot], [txt, chatbot])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://25e351ec7ec4531056.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
